In [1]:
import random
import nacl.encoding
import nacl.hash

from utils import *
from ore import *
from bplustree import *
from node import *
from read_data import *
from dtree import *

In [2]:
def test_ore_encoding_ore():
    ore = ORE(key='123')
    A = sorted([random.randint(-434531, 43454) for _ in range(100)])
    B = [scale_val(x) for x in A]
    C = [ore.encode(x) for x in B]
    co = Coordinator()
    co.receive_keys(C[:50], C[50:])
    co.mp_map.update(co.do_map)
    d = co.mp_map.copy()

    for i in range(len(C)):
        for j in range(i):
            x, y = OREncoding(d[C[j].x]), OREncoding(d[C[i].x])
            a = ore.compare(C[j], C[i])
            b = ore.compare(x, y)
            if a != -1 or a != b:
                print(i, j)

In [3]:
def simulate():
    co = Coordinator()
    do = DataOwner()
    mp = ModelProvider()
    mp.train_model()

    key_0 = str(random.randint(0, 1000))
    do.ore.key = key_0
    mp.ore.key = key_0
    do_encs = do.get_encodings()
    mp_encs = mp.get_encodings()
    co.receive_keys(mp_arr=mp_encs, do_arr=do_encs)

    mp.receive_map(co.mp_map)
    do.receive_map(co.do_map)

    enc_dataset = []
    for sample in do.dataset:
        _s = sample[:]
        for i in range(len(_s)-1):
            _v = do.ore.encode(scale_val(_s[i])).x
            _s[i] = OREncoding(do.ore_map[_v])
        enc_dataset.append(_s)

    ys = [x[-1] for x in enc_dataset]
    pred = mp.model.predict(enc_dataset)
    print(f'ACC: {accuracy_score(pred, ys) * 100:.2f}%')

# simulate()

MP model score: 97.33%
All data inserted into tree.
All data mapped to encoding.
All val in tree are mapped to OREncoding.
ACC: 97.33%


In [12]:
import time


co = Coordinator()
do = DataOwner()
mp = ModelProvider()
mp.train_model()

key_0 = str(random.randint(0, 1000))
do.ore.key = key_0
mp.ore.key = key_0
do_encs = do.get_encodings()
mp_encs = mp.get_encodings()

st = time.time()
co.receive_keys(mp_arr=mp_encs, do_arr=do_encs)
tp = (time.time() - st) * 1000
print(f'{tp/len(mp.dataset):.3f}ms')

# mp.receive_map(co.mp_map)
# do.receive_map(co.do_map)

# enc_dataset = []
# for sample in do.dataset:
#     _s = sample[:]
#     for i in range(len(_s)-1):
#         _v = do.ore.encode(scale_val(_s[i])).x
#         _s[i] = OREncoding(do.ore_map[_v])
#     enc_dataset.append(_s)

# ys = [x[-1] for x in enc_dataset]
# pred = mp.model.predict(enc_dataset)

MP model score: 97.33%
All data inserted into tree.
All data mapped to encoding.
0.790ms
